# Calculate Q-Rookley

- Input data "20172022_processed_1_3_4.csv", from 2017/7/1 to 2022/12/31, same as used in Figlewski

- Use 500 rolling window for P density (when calculating EPK)

- Use logreturn instead of log+1

- Use 
```
r.source('Q_Rookley_main_3.R')
```
and
```
'EPK_library_1.r'
```
to calculate Q_Rookley, along with 
```
get_btc_prices_2015_2022
```
for physical density.

- Interest rate: 0

- Moneyness range is [min moneyness, max moneyness]

- Bandwidth 0.3

- tau=32

In [2]:
import platform
plat = platform.system()
import locale
if plat == 'Darwin':
    print('Using Macos Locale')
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
else:
    locale.setlocale(locale.LC_ALL, 'en_US.utf8')
import datetime
import pandas as pd
import sys
import matplotlib.pyplot as plt
import csv
import gc
import math
import numpy as np
import os
import pdb
import pickle
import statsmodels.api as sm
import time


import rpy2.robjects.packages as rpackages
from rpy2 import robjects

from scipy import stats
from scipy.integrate import simps
from scipy import interpolate

from src.helpers import save_dict, load_dict
from src.brc import BRC
from src.strategies import IronCondor
from src.quandl_data import get_btc_prices_2015_2022

from numpy.lib.function_base import insert

from statsmodels.nonparametric.kernel_regression import KernelReg as nadarajawatson

from sklearn.neighbors import KernelDensity

from pathlib import Path

def filter_sub(_df,  mindate, maxdate, tau):
    # Subset

    # Only calls, tau in [0, 0.25] and fix one day (bc looking at intra day here)
    # (_df['is_call'] == 1) & --> Dont need to filter for calls because that happens in the R script
    # Also need to consider put-call parity there
    sub = _df[(_df['date'] >= mindate) & (_df['date'] <= maxdate) & (_df['moneyness'] >= 0.5) & (_df['moneyness'] < 2) &(_df['mark_iv'] > 0)]# &

    if tau > 0:
        sub = sub[sub['tau'] == tau]

    nrows = sub.shape[0]
    if nrows == 0:
        raise(ValueError('Sub is empty'))
   
    sub['moneyness'] = round(sub['moneyness'], 3)
    sub['underlying_price'] = round(sub['underlying_price'], 2)

    sub = sub.drop_duplicates()
    
    print(sub.describe())

    if nrows > 50000:
        print('large df, subsetting')
        sub = sub.sample(10000)
        print(sub.describe())

    return sub

def prepare_confidence_band_data(df):

            # Old
                # Save for Confidence Band 
                    # Column 1 - Day
                    # Column 2 - Month
                    # Column 3 - Year
                    # Column 4 - Call Indicator
                    # Column 5 - Time Maturity in Days
                    # Column 6 - Strike
                    # Column 7 - IV
                    # Column 8 - Spot
                    # Column 9 - Risk Free Rate
                    # Column 10 - 
                    # Column 11 - Forward
                # 3	1	1997	0	49	3025	165.8	2861.706	0.031465	0.150026951	2863.26
                # Day Month Year Put Days Strike IV     Spot         Risk Free    Tau in Days (255)       Forward

    """
    # Mapping Table for Input

    # Column ... X
    # 1 - date
    # 2 - IV
    # 3 - 1 for Call, 0 for Put
    # 4 - Tau 
    # 5 - Strike
    # 6 - assigned meanwhile as put2call[,6]+ put2call[,7] -  put2call[,5]*exp(- mean(put2call[,8])* tau);
    # 7 - Spot
    # 8 - Probably Interest Rate
    # 9 - assigned meanwhile Moneyness: Spot/Strike
    """
    b = df[['date', 'mark_iv', 'putcall', 'tau', 'K', 'underlying_price', 'interest_rate', 'moneyness', 'instrument_price']]
    b['placeholder'] = 0
    b = b.reset_index(drop=True)
    # b['interest_rate'] = 0.00
    b['forward'] = 1

    # Reorder
    df_c = b[['date', 'mark_iv', 'putcall', 'tau', 'K', 'instrument_price', 'underlying_price', 'interest_rate', 'moneyness']]

    unique_maturities = df.days_to_maturity.unique()[0]
    fname = str('confidence_band_input_') + str(unique_maturities) + str('.csv')
    df_c.to_csv(fname, index = False)
    return fname

def bootstrap(conf_fname, phys_fname, rdate, tau, simmethod, r, out_dir, bw, lower, upper):
                            
    try:
        # MAKE SURE TO USE DIFFERENT 2ND INPUT FOR SP500!!
        moneyness, spd, epk, epk_lo, epk_up, logret, spd_logret, epk_logret, epk_logret_lo, epk_logret_up, epk_BS, tau, volas, spd_lo, spd_up, cdf_m, cdf_ret= r.bootstrap_epk(robjects.StrVector([conf_fname]), 
                                      robjects.StrVector([phys_fname]),
                                      robjects.StrVector(rdate),
                                      robjects.StrVector([out_dir]),
                                      robjects.FloatVector([bw]),
                                      robjects.FloatVector([lower]),
                                      robjects.FloatVector([upper])
                                     )
        # Can recover PD here as SPD/EPK
        spd_df = pd.DataFrame({'m': moneyness,
                               'spdy': spd,
                               'spdy_up': spd_up,
                               'spdy_dn': spd_lo,
                               'epk': epk,
                               'ret': logret,
                               'spd_ret': spd_logret,
                               'cb_ret_up': epk_logret_up,
                               'cb_ret_dn': epk_logret_lo,
                               'epk_ret': epk_logret,
                               'volatility': volas,
                               'epk_up': epk_up,
                               'epk_dn': epk_lo,
                               'cdf_m': cdf_m,
                               'cdf_ret': cdf_ret})

        return spd_df
    
    except Exception as e:
        print('Sim failed, proceeding with the next one... - ', simmethod)
        print('exception: ', e)
        
# Initiate R Objects
base = rpackages.importr("base") 
r = robjects.r
r.source('src/confidence_bands.R')
r.source('src/Q_Rookley_main_3.R')

# BTC prices
tdat = get_btc_prices_2015_2022()
tdat[::-1].to_csv('Data/BTC_USD_Quandl_2015_2022.csv', index = False)

bw_range = np.array([0.3])
r_bandwidth = bw_range[0]

simmethod = ['oldschool']

df = pd.read_csv('Data/20172022_processed_1_3_4.csv')
interest_rate_data = pd.read_csv('Data/IR_daily.csv')
interest_rate_data = interest_rate_data.rename(columns={'index': 'date','DTB3': 'interest_rate'})

df1=df[df['tau']==32]
dates=df1['date'].unique()
#df1=df[df['date']>=dates[9]]
#dates = df1['date'].unique()

dates1 = dates

error_i = []
for i0 in range(dates1.size):
    df2 = df1[df1['date']==dates1[i0]]
    
    curr_day = datetime.datetime.strptime(dates1[i0],'%Y-%m-%d')
    curr_day_starttime = curr_day.replace(hour = 0, minute = 0, second = 0, microsecond = 0)
    curr_day_endtime = curr_day.replace(hour = 23, minute = 59, second = 59, microsecond = 0)
    print(curr_day)

    df3 = pd.merge(df2, interest_rate_data, on='date', how='left')
    

    df3['interest_rate'] = 0
    df3['days_to_maturity'] = df3['tau']
    df3['tau'] = df3['tau']/365
    df3['underlying_price'] = df3['BTC_price']
    df3['instrument_price'] = df3['option_price']
    df3['putcall'][df3['putcall']=='C']=1
    df3['putcall'][df3['putcall']=='P']=0
    
    df_1 = df3[['date','BTC_price', 'K', 'interest_rate', 'tau', 'putcall', 'IV',  'moneyness',
                'days_to_maturity',  'underlying_price', 'instrument_price']]
    df_1['mark_iv']=df_1['IV']/100
    
    df_1.to_csv('Results/option_' + datetime.datetime.strftime(curr_day_starttime, '%Y%m%d') + '.csv')
    
    conf_fname = prepare_confidence_band_data(df_1)
    
    rdate = base.as_Date(curr_day.strftime('%Y-%m-%d'))
    rdate_f = base.format(rdate, '%Y-%m-%d')
    
    tau = df['tau'].unique()[0]
    
    spd_btc = bootstrap(conf_fname, 'Data/BTC_USD_Quandl_2015_2022.csv', rdate_f, tau, simmethod, r, 
                                     'Results/Rookley_Q_CB_1722_1_2_5_1_ttm32', r_bandwidth, df_1['moneyness'].min(), df_1['moneyness'].max())
    
    try :
        
        spd_btc.to_csv('Results/Rookley_Q_CB_1722_1_2_5_1_ttm32/btc_pk_'  + str(curr_day_starttime.__format__('%Y-%m-%d')) +'_bw_' + str(r_bandwidth) +  '.csv')
        
        print(spd_btc)
        
    except Exception as e:
        error_i.append(i0)

Using Macos Locale
2017-08-28 00:00:00


/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_26126/1863764405.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_26126/1863764405.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_26126/1863764405.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] "bandwidth: 0.3"


R[write to console]: Error in (function (f, ts, currdate, out_dir, bandwidth, mon_low, mon_up)  : 
  Too few data points!

R[write to console]: In addition: 
R[write to console]: There were 50 or more warnings (use warnings() to see the first 50)
R[write to console]: 



Sim failed, proceeding with the next one... -  ['oldschool']
exception:  Error in (function (f, ts, currdate, out_dir, bandwidth, mon_low, mon_up)  : 
  Too few data points!

2017-11-27 00:00:00
[1] "bandwidth: 0.3"
[1] "test no arbitrage condition"


...

[1] 44.694713  4.722982 66.258718
                [,1]      [,2]         [,3]         [,4]
sigmas -0.6241100474 1.1342094 -1.384021516  1.544121502
       -0.6112392903 1.1248826 -1.375959581  1.567596226
       -0.5985320867 1.1155974 -1.367661501  1.590699720
       -0.5859843321 1.1063558 -1.359129500  1.613424626
       -0.5735920746 1.0971597 -1.350365926  1.635763766
       -0.5613515072 1.0880110 -1.341373256  1.657710150
       -0.5492589612 1.0789117 -1.332154091  1.679256984
       -0.5373108995 1.0698636 -1.322711157  1.700397683
       -0.5255039101 1.0608686 -1.313047300  1.721125872
       -0.5138347005 1.0519287 -1.303165488  1.741435402
       -0.5023000922 1.0430457 -1.293068803  1.761320348
       -0.4908970153 1.0342215 -1.282760443  1.780775027
       -0.4796225040 1.0254579 -1.272243719  1.799793992
       -0.4684736914 1.0167566 -1.261522050  1.818372050
       -0.4574478055 1.0081197 -1.250598959  1.836504259
       -0.4465421652 0.9995487 -1.239478074  1.8541859

        0.4195037542 0.6801151  0.364679951  0.863145823
        0.4240554943 0.6827936  0.370343903  0.837970186
        0.4285866099 0.6855101  0.375809506  0.812611866
        0.4330972871 0.6882632  0.381075158  0.787071586
        0.4375877093 0.6910513  0.386139260  0.761350056
        0.4420580578 0.6938730  0.391000217  0.735447972
        0.4465085112 0.6967266  0.395656437  0.709366019
        0.4509392458 0.6996106  0.400106329  0.683104873
        0.4553504355 0.7025235  0.404348310  0.656665202
        0.4597422520 0.7054637  0.408380797  0.630047668
        0.4641148648 0.7084294  0.412202212  0.603252929
        0.4684684411 0.7114193  0.415810981  0.576281636
        0.4728031459 0.7144315  0.419205536  0.549134443
        0.4771191421 0.7174645  0.422384312  0.521812000
        0.4814165905 0.7205167  0.425345748  0.494314960
        0.4856956498 0.7235862  0.428088292  0.466643977
        0.4899564767 0.7266716  0.430610394  0.438799712
        0.4941992261 0.7297710 

/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_26126/1863764405.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_26126/1863764405.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_26126/1863764405.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] 44.694713  4.722982 66.258718
               [,1]      [,2]         [,3]         [,4]
sigmas -0.656978861 1.1442372 -2.112567946  4.240620113
       -0.642062917 1.1281365 -2.082758762  4.237854908
       -0.627366192 1.1122456 -2.052826707  4.234304735
       -0.612882336 1.0965665 -2.022781446  4.229975065
       -0.598605270 1.0811011 -1.992632595  4.224871556
       -0.584529173 1.0658512 -1.962389717  4.219000050
       -0.570648466 1.0508187 -1.932062321  4.212366568
       -0.556957798 1.0360050 -1.901659854  4.204977306
       -0.543452035 1.0214119 -1.871191700  4.196838633
       -0.530126250 1.0070407 -1.840667175  4.187957083
       -0.516975709 0.9928927 -1.810095525  4.178339356
       -0.503995863 0.9789692 -1.779485921  4.167992308
       -0.491182336 0.9652713 -1.748847458  4.156922953
       -0.478530921 0.9518001 -1.718189149  4.145138452
       -0.466037566 0.9385566 -1.687519922  4.132646112
       -0.453698371 0.9255415 -1.656848622  4.119453382
       -0.4415

        0.494856582 0.7158801  0.497768996 -0.146535289
        0.499594949 0.7196284  0.497346020 -0.175042210
        0.504310969 0.7233734  0.496752939 -0.203194533
        0.509004852 0.7271142  0.495993967 -0.230989768
        0.513676806 0.7308499  0.495073319 -0.258425600
        0.518327034 0.7345796  0.493995215 -0.285499891
        0.522955738 0.7383025  0.492763871 -0.312210677
        0.527563116 0.7420177  0.491383503 -0.338556170
        0.532149363 0.7457246  0.489858319 -0.364534754
        0.536714672 0.7494224  0.488192518 -0.390144988
        0.541259234 0.7531105  0.486390289 -0.415385604
        0.545783236 0.7567882  0.484455805 -0.440255504
        0.550286863 0.7604549  0.482393225 -0.464753760
        0.554770299 0.7641102  0.480206685 -0.488879612
        0.559233724 0.7677535  0.477900304 -0.512632467
        0.563677315 0.7713844  0.475478174 -0.536011897
        0.568101247 0.7750024  0.472944362 -0.559017636
        0.572505695 0.7786073  0.470302906 -0.58

In [3]:
print(error_i)

[0, 5, 6, 10, 15]
